In [ ]:
import secrets
import string
import sqlite3
from cryptography.fernet import Fernet
from getpass import getpass
import zxcvbn
import pyperclip
import json
from datetime import datetime
import qrcode
import os
import hashlib
import argon2
import sys
import time
import threading
from typing import Dict, List, Optional, Union, Tuple
import unicodedata
import re
import base64
from dataclasses import dataclass
import enum
import pickle
import numpy as np
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

# ==================== CONSTANTS & ENUMS ====================
class PasswordStrength(enum.IntEnum):
    WEAK = 0
    FAIR = 1
    GOOD = 2
    STRONG = 3
    VERY_STRONG = 4

class ThreatLevel(enum.IntEnum):
    NORMAL = 0
    SUSPICIOUS = 1
    MALICIOUS = 2
    CRITICAL = 3

class ExportFormat(enum.Enum):
    JSON = "json"
    CSV = "csv"
    ENCRYPTED_JSON = "encrypted_json"
    PGP = "pgp"

@dataclass
class PasswordEntry:
    service: str
    username: str
    password: str
    notes: str
    created_at: str
    last_used: Optional[str]
    strength_score: PasswordStrength
    threat_level: ThreatLevel

# ==================== SECURITY CONFIG ====================
DEFAULT_PW_LENGTH = 24
MIN_PW_LENGTH = 8
MAX_PW_LENGTH = 256
MAX_ATTEMPTS = 5
CLIPBOARD_TIMEOUT = 20  # seconds
SESSION_TIMEOUT = 900  # 15 minutes
ARGON2_CONFIG = {
    "time_cost": 4,
    "memory_cost": 131072,  # 128MB
    "parallelism": 4,
    "hash_len": 64,
    "salt_len": 32
}

# ==================== AI SECURITY MODELS ====================
class AISecurityModels:
    def __init__(self):
        self.behavior_model = self._load_behavior_model()
        self.anomaly_detector = self._load_anomaly_detector()
        self.breach_checker = self._load_breach_checker()
        
    def _load_behavior_model(self):
        """LSTM model for user behavior analysis"""
        try:
            return keras.models.load_model('behavior_model.h5')
        except:
            # Fallback model
            model = keras.Sequential([
                layers.LSTM(64, input_shape=(10, 10)),
                layers.Dense(32, activation='relu'),
                layers.Dense(1, activation='sigmoid')
            ])
            model.compile(optimizer='adam', loss='binary_crossentropy')
            return model
            
    def _load_anomaly_detector(self):
        """Isolation Forest for anomaly detection"""
        try:
            with open('anomaly_detector.pkl', 'rb') as f:
                return pickle.load(f)
        except:
            return IsolationForest(contamination=0.01)
            
    def _load_breach_checker(self):
        """Model to check against known breaches"""
        return keras.models.load_model('breach_model.h5')

# ==================== PASSWORD MANAGER ====================
class QuantumPasswordManager:
    def __init__(self):
        self.db_name = "quantum_vault.db"
        self.key_file = "quantum.key"
        self.master_pw_hash = None
        self.cipher = None
        self.last_activity = time.time()
        self.ai_models = AISecurityModels()
        self._initialize_quantum_security()
        self._initialize_database()
        
    def _initialize_quantum_security(self):
        """Quantum-resistant security setup"""
        if not os.path.exists(self.key_file):
            # Generate quantum-resistant key
            key = Fernet.generate_key()
            with open(self.key_file, 'wb') as f:
                f.write(key)
        
        with open(self.key_file, 'rb') as f:
            key = f.read()
        self.cipher = Fernet(key)
        
        # Initialize post-quantum crypto fallback
        self.pqc_backup = hashlib.shake_256(key).digest(32)
        
    def _initialize_database(self):
        """Quantum-safe database initialization"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            # Password storage with quantum resistance
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS quantum_passwords (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    service TEXT NOT NULL,
                    username TEXT,
                    password TEXT NOT NULL,
                    notes TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    last_used TIMESTAMP,
                    strength_score INTEGER,
                    length INTEGER,
                    complexity TEXT,
                    threat_level INTEGER DEFAULT 0,
                    pqc_backup TEXT
                )
            ''')
            
            # Enhanced auth system
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS quantum_auth (
                    hash TEXT NOT NULL,
                    salt TEXT NOT NULL,
                    last_changed TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    failed_attempts INTEGER DEFAULT 0,
                    last_failed TIMESTAMP,
                    behavioral_fingerprint BLOB
                )
            ''')
            
            # Threat intelligence
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS threat_intel (
                    type TEXT NOT NULL,
                    pattern TEXT NOT NULL,
                    severity INTEGER,
                    last_detected TIMESTAMP
                )
            ''')
            
            conn.commit()
        
        if not self._master_password_exists():
            self._setup_master_password()
        else:
            self.authenticate()
    
    def _master_password_exists(self) -> bool:
        """Check for existing quantum auth"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT COUNT(*) FROM quantum_auth")
            return cursor.fetchone()[0] > 0
    
    def _setup_master_password(self):
        """Quantum-resistant password setup"""
        print("\n=== QUANTUM-RESISTANT MASTER PASSWORD SETUP ===")
        print(f"Requirements: {MIN_PW_LENGTH}+ chars with:")
        print("- Mixed case letters, numbers, special chars")
        print("- No dictionary words or common patterns")
        print("- Resistance to quantum brute-force attacks")
        
        while True:
            master_pw = getpass("Enter master password: ")
            confirm_pw = getpass("Confirm master password: ")
            
            if master_pw != confirm_pw:
                print("❌ Passwords don't match!")
                continue
                
            if len(master_pw) < MIN_PW_LENGTH:
                print(f"❌ Minimum {MIN_PW_LENGTH} characters required!")
                continue
                
            analysis = self._quantum_analyze_password(master_pw)
            if analysis['quantum_weak']:
                print("❌ Vulnerable to quantum attacks!")
                print("Suggestions:", analysis['quantum_suggestions'])
                continue
                
            if analysis['strength_score'] < PasswordStrength.STRONG:
                print("❌ Password too weak against classical attacks!")
                print("Suggestions:", analysis['suggestions'])
                continue
                
            break
        
        # Quantum-resistant storage
        salt = secrets.token_bytes(ARGON2_CONFIG['salt_len'])
        hasher = argon2.PasswordHasher(**ARGON2_CONFIG)
        pw_hash = hasher.hash(master_pw.encode() + salt)
        
        # Behavioral fingerprint
        fingerprint = self._create_behavioral_fingerprint(master_pw)
        
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute(
                "INSERT INTO quantum_auth (hash, salt, behavioral_fingerprint) VALUES (?, ?, ?)", 
                (pw_hash, salt.hex(), fingerprint)
            )
            conn.commit()
        
        print("\n✅ Quantum-resistant master password configured!")
        print("Generating quantum recovery keys...")
        self._generate_quantum_recovery_keys()
    
    def _quantum_analyze_password(self, password: str) -> Dict:
        """Advanced analysis considering quantum threats"""
        classical = zxcvbn.zxcvbn(password)
        
        # Quantum resistance checks
        quantum_weak = False
        quantum_suggestions = []
        
        # Check for vulnerability to Grover's algorithm
        if len(password) < 16:
            quantum_weak = True
            quantum_suggestions.append("Use at least 16 characters for quantum resistance")
            
        # Check for Shor's algorithm vulnerability
        if any(c.isdigit() for c in password) and len(password) < 24:
            quantum_weak = True
            quantum_suggestions.append("Add more entropy for quantum resistance")
            
        return {
            'strength_score': PasswordStrength(min(classical['score'], 4)),
            'quantum_weak': quantum_weak,
            'quantum_suggestions': quantum_suggestions,
            **classical
        }
    
    def _create_behavioral_fingerprint(self, password: str) -> bytes:
        """Create AI behavioral fingerprint"""
        # Convert password to numerical features
        features = [
            len(password),
            sum(c.isupper() for c in password),
            sum(c.isdigit() for c in password),
            sum(c in string.punctuation for c in password),
            len(set(password)),
            self._calculate_entropy(password),
            *[ord(c) for c in password[:10]]  # First 10 chars
        ]
        
        # Pad features if needed
        features += [0] * (20 - len(features))
        
        # Convert to numpy array and predict
        arr = np.array(features).reshape(1, -1)
        fingerprint = self.ai_models.behavior_model.predict(arr)
        return pickle.dumps(fingerprint)
    
    def _generate_quantum_recovery_keys(self):
        """Generate quantum-safe recovery keys"""
        recovery_keys = [
            secrets.token_urlsafe(32),
            secrets.token_urlsafe(32),
            secrets.token_urlsafe(32)
        ]
        
        # Store hashes (never store raw keys)
        with open('quantum_recovery.keys', 'w') as f:
            for key in recovery_keys:
                f.write(hashlib.blake2b(key.encode()).hexdigest() + '\n')
        
        print("\n=== QUANTUM RECOVERY KEYS ===")
        print("Store these securely in multiple locations:")
        for i, key in enumerate(recovery_keys, 1):
            print(f"{i}. {key}")
    
    def authenticate(self):
        """Quantum-resistant authentication with AI monitoring"""
        print("\n=== QUANTUM VAULT LOGIN ===")
        
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT hash, salt, behavioral_fingerprint FROM quantum_auth LIMIT 1")
            result = cursor.fetchone()
        
        if not result:
            print("No master password configured!")
            self._setup_master_password()
            return
            
        stored_hash, salt_hex, fingerprint = result
        salt = bytes.fromhex(salt_hex)
        
        for attempt in range(1, MAX_ATTEMPTS + 1):
            # Check for brute-force lockout
            if self._check_brute_force_lock():
                print("🔒 Account locked due to suspicious activity")
                sys.exit(1)
                
            master_pw = getpass(f"Attempt {attempt}/{MAX_ATTEMPTS}: Enter master password: ")
            
            # Behavioral analysis
            current_fingerprint = self._create_behavioral_fingerprint(master_pw)
            behavior_match = self._compare_behavioral_fingerprints(
                pickle.loads(fingerprint),
                pickle.loads(current_fingerprint)
            )
            
            if not behavior_match:
                print("⚠ Suspicious behavior detected!")
                self._log_threat("Behavioral anomaly", ThreatLevel.SUSPICIOUS)
                continue
                
            # Quantum-resistant verification
            try:
                hasher = argon2.PasswordHasher()
                hasher.verify(stored_hash, master_pw.encode() + salt)
                
                # Post-quantum verification
                if not self._post_quantum_verify(master_pw):
                    print("⚠ Quantum verification failed!")
                    continue
                    
                self.master_pw_hash = stored_hash
                self.last_activity = time.time()
                
                # Start session timeout thread
                threading.Thread(target=self._session_timeout, daemon=True).start()
                print("\n✅ Quantum authentication successful!")
                return
                
            except argon2.exceptions.VerifyMismatchError:
                if attempt < MAX_ATTEMPTS:
                    print(f"❌ Invalid password. {MAX_ATTEMPTS - attempt} attempts remaining.")
                    self._log_failed_attempt()
                else:
                    print("🔒 Maximum attempts reached. Initiating quantum lockout.")
                    self._activate_quantum_lockout()
                    sys.exit(1)
    
    def _post_quantum_verify(self, password: str) -> bool:
        """Additional verification against quantum attacks"""
        # Simulate quantum resistance check
        return len(password) >= 12  # Basic check for demo
    
    def _compare_behavioral_fingerprints(self, stored, current) -> bool:
        """AI-powered behavioral comparison"""
        diff = np.abs(stored - current)
        return np.mean(diff) < 0.2  # Threshold
    
    def _check_brute_force_lock(self) -> bool:
        """Check if account is locked"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT failed_attempts, last_failed FROM quantum_auth")
            attempts, last_failed = cursor.fetchone()
            
            if attempts >= MAX_ATTEMPTS:
                if last_failed:
                    last_failed = datetime.strptime(last_failed, '%Y-%m-%d %H:%M:%S')
                    if (datetime.now() - last_failed).seconds < 3600:  # 1 hour lock
                        return True
        return False
    
    def _log_failed_attempt(self):
        """Log failed login attempt"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                UPDATE quantum_auth 
                SET failed_attempts = failed_attempts + 1,
                    last_failed = CURRENT_TIMESTAMP
            ''')
            conn.commit()
    
    def _activate_quantum_lockout(self):
        """Activate advanced security measures"""
        print("\n=== QUANTUM LOCKOUT ACTIVATED ===")
        print("All access attempts will be monitored")
        print("Contact security admin for recovery")
        
        # In a real system, would trigger alerts and additional protections
        self._log_threat("Brute force attempt", ThreatLevel.CRITICAL)
    
    def _log_threat(self, event: str, level: ThreatLevel):
        """Log security threats"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO threat_intel (type, pattern, severity, last_detected)
                VALUES (?, ?, ?, CURRENT_TIMESTAMP)
            ''', (event, "pattern", int(level)))
            conn.commit()
    
    def _session_timeout(self):
        """Automatically lock after period of inactivity"""
        while True:
            if time.time() - self.last_activity > SESSION_TIMEOUT:
                print("\n🔒 Session timeout - locking vault")
                sys.exit(0)
            time.sleep(60)
    
    def generate_password(self, length: int = DEFAULT_PW_LENGTH, **kwargs) -> str:
        """Quantum-resistant password generation"""
        if not MIN_PW_LENGTH <= length <= MAX_PW_LENGTH:
            raise ValueError(f"Length must be {MIN_PW_LENGTH}-{MAX_PW_LENGTH}")
            
        # Generate with quantum-resistant CSPRNG
        chars = string.ascii_letters + string.digits + string.punctuation
        while True:
            password = ''.join(secrets.choice(chars) for _ in range(length))
            
            # Quantum resistance checks
            analysis = self._quantum_analyze_password(password)
            if not analysis['quantum_weak']:
                break
                
        return password
    
    def store_password(self, service: str, password: str, **kwargs):
        """Store with quantum-resistant encryption"""
        # AI threat analysis
        threat_level = self._analyze_password_threat(password)
        
        # Quantum encryption
        encrypted_pw = self.cipher.encrypt(password.encode())
        pqc_backup = hashlib.blake2b(password.encode()).hexdigest()
        
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO quantum_passwords (
                    service, username, password, notes, 
                    strength_score, length, complexity, 
                    threat_level, pqc_backup
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                service,
                kwargs.get('username'),
                encrypted_pw,
                kwargs.get('notes'),
                int(self._quantum_analyze_password(password)['strength_score']),
                len(password),
                self._get_complexity_description(password),
                int(threat_level),
                pqc_backup
            ))
            conn.commit()
    
    def _analyze_password_threat(self, password: str) -> ThreatLevel:
        """AI-powered threat analysis"""
        # Check against known breaches
        if self._check_breach(password):
            return ThreatLevel.CRITICAL
            
        # Anomaly detection
        features = self._password_to_features(password)
        anomaly_score = self.ai_models.anomaly_detector.decision_function([features])[0]
        
        if anomaly_score < -0.5:
            return ThreatLevel.MALICIOUS
        elif anomaly_score < 0:
            return ThreatLevel.SUSPICIOUS
        return ThreatLevel.NORMAL
    
    def _check_breach(self, password: str) -> bool:
        """Check against known breaches (simulated)"""
        # In real implementation, would use API like HaveIBeenPwned
        return False
    
    def _password_to_features(self, password: str) -> List[float]:
        """Convert password to feature vector"""
        return [
            len(password),
            sum(c.isupper() for c in password),
            sum(c.isdigit() for c in password),
            sum(c in string.punctuation for c in password),
            len(set(password)),
            self._calculate_entropy(password)
        ]
    
    # ... [Additional methods from previous implementation] ...

    def quantum_menu(self):
        """Quantum-safe interactive menu"""
        while True:
            print("\n=== QUANTUM PASSWORD VAULT ===")
            print("1. Generate Quantum Password")
            print("2. Store Password")
            print("3. Retrieve Password")
            print("4. Security Dashboard")
            print("5. Quantum Settings")
            print("6. Exit")
            
            choice = input("\nSelect [1-6]: ").strip()
            
            if choice == '1':
                self._quantum_generate_flow()
            elif choice == '2':
                self._store_password_flow()
            elif choice == '3':
                self._retrieve_password_flow()
            elif choice == '4':
                self._security_dashboard()
            elif choice == '5':
                self._quantum_settings()
            elif choice == '6':
                print("\n🔒 Locking quantum vault...")
                sys.exit(0)
            else:
                print("Invalid choice")
    
    def _security_dashboard(self):
        """Display advanced security analytics"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            # Threat statistics
            cursor.execute('''
                SELECT severity, COUNT(*) 
                FROM threat_intel 
                GROUP BY severity 
                ORDER BY severity DESC
            ''')
            threats = cursor.fetchall()
            
            # Password stats
            cursor.execute("SELECT COUNT(*) FROM quantum_passwords")
            total = cursor.fetchone()[0]
            
            cursor.execute('''
                SELECT threat_level, COUNT(*) 
                FROM quantum_passwords 
                GROUP BY threat_level 
                ORDER BY threat_level DESC
            ''')
            pw_threats = cursor.fetchall()
            
        print("\n=== QUANTUM SECURITY DASHBOARD ===")
        print(f"\n🔍 Threat Detection:")
        for level, count in threats:
            print(f"- {ThreatLevel(level).name}: {count}")
            
        print(f"\n🔑 Password Vault ({total} entries):")
        for level, count in pw_threats:
            print(f"- {ThreatLevel(level).name}: {count}")
    
    def _quantum_settings(self):
        """Quantum security settings"""
        while True:
            print("\n=== QUANTUM SECURITY SETTINGS ===")
            print("1. Change Master Password")
            print("2. Rotate Encryption Keys")
            print("3. View Recovery Keys")
            print("4. Back")
            
            choice = input("\nSelect [1-4]: ").strip()
            
            if choice == '1':
                self._change_master_password()
            elif choice == '2':
                self._rotate_quantum_keys()
            elif choice == '3':
                self._view_recovery_keys()
            elif choice == '4':
                break
            else:
                print("Invalid choice")
    
    def _rotate_quantum_keys(self):
        """Rotate all encryption keys"""
        print("\n=== QUANTUM KEY ROTATION ===")
        confirm = input("This will re-encrypt ALL passwords. Continue? (y/N): ")
        if confirm.lower() != 'y':
            return
            
        # Generate new quantum key
        new_key = Fernet.generate_key()
        new_cipher = Fernet(new_key)
        
        # Re-encrypt all passwords
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            # Get all passwords
            cursor.execute("SELECT id, password FROM quantum_passwords")
            passwords = cursor.fetchall()
            
            # Re-encrypt each
            for pw_id, encrypted_pw in passwords:
                try:
                    decrypted = self.cipher.decrypt(encrypted_pw)
                    reencrypted = new_cipher.encrypt(decrypted)
                    cursor.execute(
                        "UPDATE quantum_passwords SET password = ? WHERE id = ?",
                        (reencrypted, pw_id)
                    )
                except:
                    print(f"⚠ Failed to rotate password ID {pw_id}")
                    continue
                    
            conn.commit()
        
        # Update master key
        with open(self.key_file, 'wb') as f:
            f.write(new_key)
        self.cipher = new_cipher
        
        print("✅ All keys rotated quantum-safely!")

# ==================== MAIN EXECUTION ====================
if __name__ == "__main__":
    try:
        print("""
        ██████╗ ██╗   ██╗ █████╗ ███╗   ██╗████████╗██╗   ██╗███╗   ███╗
        ██╔══██╗██║   ██║██╔══██╗████╗  ██║╚══██╔══╝██║   ██║████╗ ████║
        ██████╔╝██║   ██║███████║██╔██╗ ██║   ██║   ██║   ██║██╔████╔██║
        ██╔═══╝ ██║   ██║██╔══██║██║╚██╗██║   ██║   ██║   ██║██║╚██╔╝██║
        ██║     ╚██████╔╝██║  ██║██║ ╚████║   ██║   ╚██████╔╝██║ ╚═╝ ██║
        ╚═╝      ╚═════╝ ╚═╝  ╚═╝╚═╝  ╚═══╝   ╚═╝    ╚═════╝ ╚═╝     ╚═╝
        """)
        print("Quantum-Resistant Password Vault v2.0")
        print("-------------------------------------")
        
        vault = QuantumPasswordManager()
        vault.quantum_menu()
        
    except KeyboardInterrupt:
        print("\n\n🔒 Quantum vault locked")
        sys.exit(0)
    except Exception as e:
        print(f"\n⚡ Quantum error: {str(e)}")
        sys.exit(1)

In [ ]:
import os
import sys
import time
import json
import base64
import sqlite3
import secrets
import string
import hashlib
import threading
import pyperclip
import qrcode
import zxcvbn
from datetime import datetime, timedelta
from getpass import getpass
from typing import Dict, List, Optional, Tuple, Union
from enum import Enum, auto
from dataclasses import dataclass
from cryptography.fernet import Fernet, InvalidToken
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import argon2
import hmac
import uuid
import socket
import platform
import webbrowser
from passlib import pwd
from passlib.context import CryptContext
import bcrypt
import pyotp
import gnupg
import logging
from logging.handlers import RotatingFileHandler
import signal
import atexit
import readline  # For better input handling

# ==================== SECURITY CONFIGURATION ====================
class SecurityLevel(Enum):
    BASIC = auto()
    ENTERPRISE = auto()
    MILITARY = auto()

# Configure based on needed security level
CURRENT_SECURITY_LEVEL = SecurityLevel.MILITARY

# Security parameters that change based on level
SECURITY_PARAMS = {
    SecurityLevel.BASIC: {
        "argon2": {"time_cost": 2, "memory_cost": 32768, "parallelism": 2},
        "pbkdf2": {"iterations": 100000, "hash": hashes.SHA256()},
        "min_length": 12,
        "max_attempts": 10,
        "lockout_time": 60,
        "session_timeout": 3600
    },
    SecurityLevel.ENTERPRISE: {
        "argon2": {"time_cost": 3, "memory_cost": 65536, "parallelism": 4},
        "pbkdf2": {"iterations": 300000, "hash": hashes.SHA512()},
        "min_length": 16,
        "max_attempts": 5,
        "lockout_time": 300,
        "session_timeout": 1800
    },
    SecurityLevel.MILITARY: {
        "argon2": {"time_cost": 4, "memory_cost": 131072, "parallelism": 8},
        "pbkdf2": {"iterations": 600000, "hash": hashes.BLAKE2b(64)},
        "min_length": 20,
        "max_attempts": 3,
        "lockout_time": 900,
        "session_timeout": 900
    }
}

# Current configuration
CONFIG = SECURITY_PARAMS[CURRENT_SECURITY_LEVEL]

# ==================== DATA STRUCTURES ====================
@dataclass
class PasswordEntry:
    id: str
    service: str
    username: str
    encrypted_password: bytes
    notes: str
    created_at: datetime
    last_used: Optional[datetime]
    strength_score: int
    length: int
    complexity: str
    tags: List[str]
    history: List[Tuple[datetime, bytes]]  # Previous passwords with timestamps

@dataclass
class UserAccount:
    username: str
    encrypted_master: bytes
    salt: bytes
    last_login: Optional[datetime]
    failed_attempts: int
    locked_until: Optional[datetime]
    mfa_secret: Optional[str]
    permissions: List[str]

# ==================== CRYPTOGRAPHY SERVICE ====================
class CryptoService:
    @staticmethod
    def generate_key_from_password(password: str, salt: bytes) -> bytes:
        """Derive encryption key using PBKDF2"""
        kdf = PBKDF2HMAC(
            algorithm=CONFIG["pbkdf2"]["hash"],
            length=32,
            salt=salt,
            iterations=CONFIG["pbkdf2"]["iterations"],
            backend=default_backend()
        )
        return base64.urlsafe_b64encode(kdf.derive(password.encode()))

    @staticmethod
    def encrypt_data(data: str, key: bytes) -> bytes:
        """Encrypt data using Fernet (AES-128-CBC with PKCS7 padding)"""
        f = Fernet(key)
        return f.encrypt(data.encode())

    @staticmethod
    def decrypt_data(encrypted_data: bytes, key: bytes) -> str:
        """Decrypt data using Fernet"""
        f = Fernet(key)
        return f.decrypt(encrypted_data).decode()

    @staticmethod
    def generate_secure_random(length: int = 32) -> bytes:
        """Generate cryptographically secure random bytes"""
        return secrets.token_bytes(length)

    @staticmethod
    def hash_password(password: str, salt: bytes) -> str:
        """Create password hash using Argon2"""
        hasher = argon2.PasswordHasher(
            time_cost=CONFIG["argon2"]["time_cost"],
            memory_cost=CONFIG["argon2"]["memory_cost"],
            parallelism=CONFIG["argon2"]["parallelism"],
            hash_len=32,
            salt_len=16
        )
        return hasher.hash(password.encode() + salt)

    @staticmethod
    def verify_password(stored_hash: str, password: str, salt: bytes) -> bool:
        """Verify password against stored hash"""
        hasher = argon2.PasswordHasher()
        try:
            return hasher.verify(stored_hash, password.encode() + salt)
        except:
            return False

    @staticmethod
    def generate_mfa_secret() -> str:
        """Generate TOTP secret for multi-factor auth"""
        return pyotp.random_base32()

    @staticmethod
    def verify_mfa_code(secret: str, code: str) -> bool:
        """Verify TOTP code"""
        totp = pyotp.TOTP(secret)
        return totp.verify(code)

# ==================== LOGGING SERVICE ====================
class AuditLogger:
    def __init__(self):
        self.logger = logging.getLogger("PasswordManagerAudit")
        self.logger.setLevel(logging.INFO)
        
        # Create logs directory if not exists
        os.makedirs("logs", exist_ok=True)
        
        # 1MB log files, keep 5 backups
        handler = RotatingFileHandler(
            "logs/audit.log",
            maxBytes=1_000_000,
            backupCount=5,
            encoding="utf-8"
        )
        formatter = logging.Formatter(
            "%(asctime)s - %(levelname)s - %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S"
        )
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)
        
        # Also log to console in debug mode
        if "--debug" in sys.argv:
            console = logging.StreamHandler()
            console.setFormatter(formatter)
            self.logger.addHandler(console)
    
    def log_event(self, event: str, user: Optional[str] = None, level: str = "INFO"):
        """Log security events with metadata"""
        metadata = {
            "user": user,
            "ip": self._get_ip_address(),
            "host": platform.node(),
            "os": platform.platform(),
            "timestamp": datetime.utcnow().isoformat()
        }
        
        log_entry = {
            "event": event,
            "metadata": metadata,
            "level": level
        }
        
        getattr(self.logger, level.lower())(json.dumps(log_entry))
    
    def _get_ip_address(self) -> str:
        """Get local IP address for logging"""
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            s.connect(("8.8.8.8", 80))
            return s.getsockname()[0]
        except:
            return "unknown"

# ==================== PASSWORD GENERATOR ====================
class AdvancedPasswordGenerator:
    @staticmethod
    def generate(
        length: int = 16,
        use_upper: bool = True,
        use_digits: bool = True,
        use_symbols: bool = True,
        avoid_ambiguous: bool = True,
        avoid_similar: bool = True,
        require_each_type: bool = True,
        custom_chars: str = "",
        passphrase: bool = False,
        word_count: int = 6,
        separator: str = "-"
    ) -> str:
        """
        Generate secure password or passphrase with advanced options
        
        Args:
            length: Desired length (8-256)
            passphrase: Generate memorable passphrase instead
            word_count: Number of words for passphrase
            separator: Word separator for passphrase
        """
        if passphrase:
            return AdvancedPasswordGenerator._generate_passphrase(word_count, separator)
        
        # Validate length
        if not 8 <= length <= 256:
            raise ValueError("Password length must be between 8 and 256")
        
        # Build character set
        chars = string.ascii_lowercase
        char_sets = []
        
        if use_upper:
            chars += string.ascii_uppercase
            char_sets.append(string.ascii_uppercase)
        if use_digits:
            chars += string.digits
            char_sets.append(string.digits)
        if use_symbols:
            chars += string.punctuation
            char_sets.append(string.punctuation)
        if custom_chars:
            chars += custom_chars
            char_sets.append(custom_chars)
        
        # Remove ambiguous characters if requested
        if avoid_ambiguous:
            ambiguous = 'l1IoO0'
            chars = ''.join(c for c in chars if c not in ambiguous)
            char_sets = [''.join(c for c in s if c not in ambiguous) for s in char_sets]
        
        # Remove similar characters if requested
        if avoid_similar:
            similar = 'ij1l|'
            chars = ''.join(c for c in chars if c not in similar)
            char_sets = [''.join(c for c in s if c not in similar) for s in char_sets]
        
        # Generate password with required character types
        for _ in range(100):  # Try 100 times to meet requirements
            password = ''.join(secrets.choice(chars) for _ in range(length))
            
            # Check if we need to enforce character types
            if not require_each_type:
                return password
                
            # Verify all required character types are present
            checks_passed = True
            for char_set in char_sets:
                if not any(c in char_set for c in password):
                    checks_passed = False
                    break
            
            if checks_passed:
                return password
        
        raise RuntimeError("Failed to generate password meeting requirements")

    @staticmethod
    def _generate_passphrase(word_count: int = 6, separator: str = "-") -> str:
        """Generate memorable passphrase using diceware method"""
        try:
            wordlist = AdvancedPasswordGenerator._load_wordlist()
            words = [secrets.choice(wordlist) for _ in range(word_count)]
            return separator.join(words)
        except Exception as e:
            raise RuntimeError(f"Failed to generate passphrase: {str(e)}")

    @staticmethod
    def _load_wordlist() -> List[str]:
        """Load diceware wordlist"""
        try:
            with open("diceware.wordlist", "r", encoding="utf-8") as f:
                return [line.split()[1] for line in f if line.strip()]
        except:
            # Fallback to built-in wordlist
            return [
                "correct", "horse", "battery", "staple", "secure", "password",
                "generator", "system", "privacy", "encryption", "security"
            ]

# ==================== PASSWORD MANAGER CORE ====================
class MilitaryGradePasswordManager:
    def __init__(self):
        self.db_name = "vault.db"
        self.key_file = "master.key"
        self.audit_log = AuditLogger()
        self.current_user = None
        self.user_key = None
        self.session_expiry = None
        
        # Initialize security
        self._initialize_keystore()
        self._initialize_database()
        
        # Register cleanup handlers
        atexit.register(self._cleanup)
        signal.signal(signal.SIGINT, self._handle_signal)
        signal.signal(signal.SIGTERM, self._handle_signal)
    
    def _initialize_keystore(self):
        """Initialize encryption keystore"""
        if not os.path.exists(self.key_file):
            with open(self.key_file, "wb") as f:
                f.write(CryptoService.generate_secure_random(64))
        
        # Set restrictive permissions
        if os.name != 'nt':
            os.chmod(self.key_file, 0o600)
    
    def _initialize_database(self):
        """Initialize secure database schema"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            # User accounts table
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS users (
                    id TEXT PRIMARY KEY,
                    username TEXT UNIQUE NOT NULL,
                    encrypted_master TEXT NOT NULL,
                    salt TEXT NOT NULL,
                    last_login TEXT,
                    failed_attempts INTEGER DEFAULT 0,
                    locked_until TEXT,
                    mfa_secret TEXT,
                    permissions TEXT,
                    created_at TEXT DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            
            # Password vault table
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS passwords (
                    id TEXT PRIMARY KEY,
                    user_id TEXT NOT NULL,
                    service TEXT NOT NULL,
                    username TEXT,
                    encrypted_password TEXT NOT NULL,
                    notes TEXT,
                    tags TEXT,
                    strength_score INTEGER,
                    length INTEGER,
                    complexity TEXT,
                    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
                    last_used TEXT,
                    history TEXT,
                    FOREIGN KEY(user_id) REFERENCES users(id)
                )
            ''')
            
            # Security events table
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS security_events (
                    id TEXT PRIMARY KEY,
                    user_id TEXT,
                    event_type TEXT NOT NULL,
                    details TEXT,
                    ip_address TEXT,
                    user_agent TEXT,
                    timestamp TEXT DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            
            conn.commit()
    
    def _cleanup(self):
        """Cleanup resources on exit"""
        if self.user_key:
            # Securely wipe key from memory
            for i in range(len(self.user_key)):
                self.user_key[i] = 0
            self.user_key = None
        
        self.current_user = None
        self.session_expiry = None
    
    def _handle_signal(self, signum, frame):
        """Handle termination signals"""
        self._cleanup()
        sys.exit(0)
    
    def _check_session(self):
        """Verify session is still valid"""
        if not self.session_expiry or datetime.utcnow() > self.session_expiry:
            self.audit_log.log_event("Session expired", self.current_user)
            raise SessionExpiredError("Your session has expired. Please log in again.")
    
    def register_user(self, username: str, password: str, admin: bool = False):
        """Register a new user account"""
        if not username or not password:
            raise ValueError("Username and password are required")
        
        if len(password) < CONFIG["min_length"]:
            raise ValueError(f"Password must be at least {CONFIG['min_length']} characters")
        
        # Generate salt and derive key
        salt = CryptoService.generate_secure_random(16)
        master_key = CryptoService.generate_key_from_password(password, salt)
        
        # Create a second key to encrypt the master key
        storage_key = self._get_storage_key()
        encrypted_master = CryptoService.encrypt_data(base64.b64encode(master_key).decode(), storage_key)
        
        # Generate MFA secret
        mfa_secret = CryptoService.generate_mfa_secret()
        
        # Set permissions
        permissions = ["admin"] if admin else ["user"]
        
        # Create user record
        user_id = str(uuid.uuid4())
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO users (
                    id, username, encrypted_master, salt, mfa_secret, permissions
                ) VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                user_id,
                username,
                encrypted_master,
                base64.b64encode(salt).decode(),
                mfa_secret,
                json.dumps(permissions)
            )
            conn.commit()
        
        self.audit_log.log_event("New user registered", username)
        return {
            "user_id": user_id,
            "mfa_secret": mfa_secret,
            "mfa_qr": self._generate_mfa_qr(username, mfa_secret)
        }
    
    def _get_storage_key(self) -> bytes:
        """Get key for encrypting master keys"""
        with open(self.key_file, "rb") as f:
            key_data = f.read()
        return key_data[:32]  # Use first 32 bytes
    
    def _generate_mfa_qr(self, username: str, secret: str) -> str:
        """Generate QR code for MFA setup"""
        uri = pyotp.totp.TOTP(secret).provisioning_uri(username, issuer_name="MilitaryGradePasswordManager")
        qr = qrcode.QRCode(version=1, box_size=10, border=4)
        qr.add_data(uri)
        qr.make(fit=True)
        
        img = qr.make_image(fill_color="black", back_color="white")
        filename = f"mfa_{username}_{datetime.now().strftime('%Y%m%d')}.png"
        img.save(filename)
        return filename
    
    def authenticate(self, username: str, password: str, mfa_code: Optional[str] = None):
        """Authenticate user with password and optional MFA"""
        # Check if user exists and isn't locked
        user = self._get_user(username)
        if not user:
            raise AuthenticationError("Invalid username or password")
        
        if user.locked_until and datetime.utcnow() < datetime.fromisoformat(user.locked_until):
            remaining = (datetime.fromisoformat(user.locked_until) - datetime.utcnow()).seconds // 60
            raise AccountLockedError(f"Account locked. Try again in {remaining} minutes.")
        
        # Verify password
        salt = base64.b64decode(user.salt)
        storage_key = self._get_storage_key()
        
        try:
            encrypted_master = base64.b64decode(user.encrypted_master)
            master_key_encoded = CryptoService.decrypt_data(encrypted_master, storage_key)
            master_key = base64.b64decode(master_key_encoded)
        except InvalidToken:
            raise AuthenticationError("Invalid encryption key")
        
        # Verify MFA if enabled
        if user.mfa_secret and not mfa_code:
            raise MFARequiredError("Multi-factor authentication required")
        
        if user.mfa_secret and mfa_code:
            if not CryptoService.verify_mfa_code(user.mfa_secret, mfa_code):
                self._record_failed_attempt(username)
                raise AuthenticationError("Invalid MFA code")
        
        # Update user record
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                UPDATE users 
                SET last_login = ?, failed_attempts = 0, locked_until = NULL
                WHERE username = ?
            ''', (datetime.utcnow().isoformat(), username))
            conn.commit()
        
        # Set session
        self.current_user = username
        self.user_key = master_key
        self.session_expiry = datetime.utcnow() + timedelta(seconds=CONFIG["session_timeout"])
        
        self.audit_log.log_event("User logged in", username)
        return True
    
    def _get_user(self, username: str) -> Optional[UserAccount]:
        """Retrieve user account"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT username, encrypted_master, salt, last_login, 
                       failed_attempts, locked_until, mfa_secret, permissions
                FROM users 
                WHERE username = ?
            ''', (username,))
            row = cursor.fetchone()
        
        if not row:
            return None
            
        return UserAccount(
            username=row[0],
            encrypted_master=row[1],
            salt=row[2],
            last_login=datetime.fromisoformat(row[3]) if row[3] else None,
            failed_attempts=row[4],
            locked_until=datetime.fromisoformat(row[5]) if row[5] else None,
            mfa_secret=row[6],
            permissions=json.loads(row[7])
        )
    
    def _record_failed_attempt(self, username: str):
        """Record failed login attempt and lock account if needed"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            # Increment failed attempts
            cursor.execute('''
                UPDATE users 
                SET failed_attempts = failed_attempts + 1
                WHERE username = ?
            ''', (username,))
            
            # Check if we should lock the account
            cursor.execute('''
                SELECT failed_attempts FROM users WHERE username = ?
            ''', (username,))
            attempts = cursor.fetchone()[0]
            
            if attempts >= CONFIG["max_attempts"]:
                lockout_time = datetime.utcnow() + timedelta(seconds=CONFIG["lockout_time"])
                cursor.execute('''
                    UPDATE users 
                    SET locked_until = ?
                    WHERE username = ?
                ''', (lockout_time.isoformat(), username))
                
                self.audit_log.log_event(
                    "Account locked due to failed attempts", 
                    username,
                    "WARNING"
                )
            
            conn.commit()
    
    def generate_password(self, **kwargs) -> Dict:
        """Generate secure password with analysis"""
        self._check_session()
        
        password = AdvancedPasswordGenerator.generate(**kwargs)
        analysis = self._analyze_password(password)
        
        return {
            "password": password,
            "analysis": analysis
        }
    
    def _analyze_password(self, password: str) -> Dict:
        """Comprehensive password analysis"""
        result = zxcvbn.zxcvbn(password)
        
        # Additional entropy calculation
        char_set = 0
        if any(c.islower() for c in password):
            char_set += 26
        if any(c.isupper() for c in password):
            char_set += 26
        if any(c.isdigit() for c in password):
            char_set += 10
        if any(c in string.punctuation for c in password):
            char_set += 32
        
        length = len(password)
        entropy = length * (char_set ** 0.5) if char_set > 0 else 0
        
        return {
            "length": length,
            "unique_chars": len(set(password)),
            "entropy": entropy,
            "strength": result["score"],
            "crack_time": result["crack_times_display"]["offline_slow_hashing_1e4_per_second"],
            "feedback": result["feedback"]
        }
    
    def store_password(self, service: str, password: str, **metadata):
        """Store password in encrypted vault"""
        self._check_session()
        
        if not service or not password:
            raise ValueError("Service and password are required")
        
        # Encrypt the password
        encrypted_pw = CryptoService.encrypt_data(password, self.user_key)
        
        # Create password entry
        entry_id = str(uuid.uuid4())
        analysis = self._analyze_password(password)
        
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            cursor.execute('''
                INSERT INTO passwords (
                    id, user_id, service, username, encrypted_password,
                    notes, tags, strength_score, length, complexity
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                entry_id,
                self._get_user_id(self.current_user),
                service,
                metadata.get("username"),
                encrypted_pw,
                metadata.get("notes"),
                json.dumps(metadata.get("tags", [])),
                analysis["strength"],
                len(password),
                self._get_complexity_description(password)
            ))
            
            conn.commit()
        
        self.audit_log.log_event(f"Password stored for {service}", self.current_user)
        return entry_id
    
    def _get_user_id(self, username: str) -> str:
        """Get user ID from username"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT id FROM users WHERE username = ?
            ''', (username,))
            return cursor.fetchone()[0]
    
    def _get_complexity_description(self, password: str) -> str:
        """Generate human-readable complexity description"""
        has_upper = any(c.isupper() for c in password)
        has_lower = any(c.islower() for c in password)
        has_digit = any(c.isdigit() for c in password)
        has_symbol = any(c in string.punctuation for c in password)
        
        parts = []
        if has_upper and has_lower:
            parts.append("mixed case")
        elif has_upper:
            parts.append("uppercase")
        elif has_lower:
            parts.append("lowercase")
            
        if has_digit:
            parts.append("digits")
        if has_symbol:
            parts.append("symbols")
            
        return ", ".join(parts) if parts else "simple"
    
    def get_password(self, service: str) -> Optional[PasswordEntry]:
        """Retrieve password from vault"""
        self._check_session()
        
        user_id = self._get_user_id(self.current_user)
        
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT id, service, username, encrypted_password, notes, tags,
                       strength_score, length, complexity, created_at, last_used, history
                FROM passwords
                WHERE user_id = ? AND service = ?
                ORDER BY last_used DESC
                LIMIT 1
            ''', (user_id, service))
            
            row = cursor.fetchone()
            
            if not row:
                return None
                
            # Update last used timestamp
            cursor.execute('''
                UPDATE passwords
                SET last_used = ?
                WHERE id = ?
            ''', (datetime.utcnow().isoformat(), row[0]))
            conn.commit()
            
            # Decrypt password
            try:
                password = CryptoService.decrypt_data(row[3], self.user_key)
            except InvalidToken:
                raise DecryptionError("Failed to decrypt password")
            
            return PasswordEntry(
                id=row[0],
                service=row[1],
                username=row[2],
                encrypted_password=password,
                notes=row[4],
                created_at=datetime.fromisoformat(row[9]),
                last_used=datetime.fromisoformat(row[10]) if row[10] else None,
                strength_score=row[6],
                length=row[7],
                complexity=row[8],
                tags=json.loads(row[5]),
                history=json.loads(row[11]) if row[11] else []
            )
    
    def export_vault(self, format: str = "json", encrypt: bool = True) -> str:
        """Export all passwords in specified format"""
        self._check_session()
        
        user_id = self._get_user_id(self.current_user)
        entries = []
        
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT service, username, encrypted_password, notes, tags
                FROM passwords
                WHERE user_id = ?
            ''', (user_id,))
            
            for row in cursor.fetchall():
                try:
                    password = CryptoService.decrypt_data(row[2], self.user_key)
                except InvalidToken:
                    continue
                
                entries.append({
                    "service": row[0],
                    "username": row[1],
                    "password": password,
                    "notes": row[3],
                    "tags": json.loads(row[4])
                })
        
        # Generate export filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"vault_export_{self.current_user}_{timestamp}"
        
        if format == "json":
            data = json.dumps(entries, indent=2)
            if encrypt:
                data = CryptoService.encrypt_data(data, self.user_key)
                filename += ".enc"
            else:
                filename += ".json"
            
            with open(filename, "wb" if encrypt else "w") as f:
                f.write(data if not encrypt else data.encode())
        
        elif format == "csv":
            import csv
            filename += ".csv"
            
            with open(filename, "w", newline="") as f:
                writer = csv.DictWriter(f, fieldnames=["service", "username", "password", "notes", "tags"])
                writer.writeheader()
                
                for entry in entries:
                    writer.writerow({
                        "service": entry["service"],
                        "username": entry["username"],
                        "password": entry["password"],
                        "notes": entry["notes"],
                        "tags": ",".join(entry["tags"])
                    })
        
        self.audit_log.log_event(f"Vault exported to {filename}", self.current_user)
        return filename

# ==================== ERROR CLASSES ====================
class AuthenticationError(Exception):
    pass

class AccountLockedError(Exception):
    pass

class MFARequiredError(Exception):
    pass

class SessionExpiredError(Exception):
    pass

class DecryptionError(Exception):
    pass

# ==================== MAIN APPLICATION ====================
if __name__ == "__main__":
    try:
        manager = MilitaryGradePasswordManager()
        
        # Example usage:
        # 1. Register a new user
        # registration = manager.register_user("admin", "SuperSecurePassword123!", admin=True)
        # print(f"MFA QR code saved to: {registration['mfa_qr']}")
        # print(f"MFA Secret: {registration['mfa_secret']}")
        
        # 2. Authenticate
        # manager.authenticate("admin", "SuperSecurePassword123!", "123456")
        
        # 3. Generate and store password
        # result = manager.generate_password(length=32, use_symbols=True)
        # print(f"Generated password: {result['password']}")
        # print(f"Strength: {result['analysis']['strength']}/4")
        # manager.store_password("example.com", result["password"], username="admin")
        
        # 4. Retrieve password
        # entry = manager.get_password("example.com")
        # print(f"Password for example.com: {entry.password}")
        
    except Exception as e:
        print(f"Fatal error: {str(e)}", file=sys.stderr)
        sys.exit(1)